## **0. Pre-Lims Code Test**

## **0.0 Install Project Deps**


In [ ]:
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-7lmp1gz_
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-7lmp1gz_
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit dcf6bde8d64e1a747673d24dde30dce195bc3cf1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-figz8y26/elegantrl_ae3b7bc3d3094417a19b9e4ed9292751
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-figz8y26/elegantrl_ae3b7bc3d3094417a19b9e4ed9292751
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 7c64395ff4c2b53892fe6f9eaa3b58947136ffc1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/1

In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/DRL/App/DRL/FinRL'
!ls

/content/drive/MyDrive/Colab Notebooks/DRL/App/DRL/FinRL
 anytrade
 binance-22-25
 condacolab_install.log
 datasets
 Fin_Opt_ERL.ipynb
 FinRL
 FinRl_app.py
 FinRL_Basic.ipynb
 FinRL_HyperparameterTuning_Optuna.ipynb
 FinRl_Mods
 FinRl_Opt.py
'FinRL_PortfolioAllocation_Explainable_DRL (1).ipynb'
 FinRL_PortfolioAllocation_Explainable_DRL.ipynb
 FinRL_PortfolioAllocation_NeurIPS_2020.ipynb
 Fin_Stock.ipynb
 GSPC
 MultiCrypto_Trading.ipynb
 result.csv
 results
 snp_500
 Stock_NeurIPS2018_SB3.ipynb
 tensorboard_log
 trained_models
 vix_daily.csv


### **0.1 Surpress Datetime Errors**

In [ ]:
import warnings
import datetime
from datetime import UTC

warnings.filterwarnings("ignore", category=DeprecationWarning)
# datetime.datetime.utcnow = lambda: datetime.datetime.now(UTC)
print("✅ Colab startup: datetime.utcnow patched.")


✅ Colab startup: datetime.utcnow patched.


### **0.2 Test Colab Input Widgets**

In [ ]:
from ipywidgets import IntText, FloatText, SelectMultiple, VBox
from IPython.display import display

# Widgets
decimal_input = FloatText(description="Decimal:")
integer_input = IntText(description="Integer:")
ticker_picker = SelectMultiple(
    options=['AAPL','AMZN','MSFT','GOOGL','TSLA','NVDA'],
    description="Tickers:"
)

# Display UI
display(VBox([decimal_input, integer_input, ticker_picker]))

# Access values
def get_inputs():
    dec = decimal_input.value
    integer = integer_input.value
    tickers = list(ticker_picker.value)
    return dec, integer, tickers

# Example usage:
dec, integer, tickers = get_inputs()
print(dec, integer, tickers)


0.0 0 []


## **1. Load Data**

In [ ]:
import itertools
import pandas as pd

df = pd.read_csv('snp_500/all_stocks_5yr.csv')
df = df.rename(columns={'Name': 'tic'})
df.columns = df.columns.str.lower()
# Ensure date column is datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# tickers_input_string = "AAPL, MSFT, GOOGL" #@param {type:"string"}
# ✅ Define 15 prominent tickers
selected_tickers = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'BRK.B',
    'JPM', 'JNJ', 'V', 'PG', 'XOM', 'UNH', 'HD'
]

# Filter data for those tickers
subset_df = df[df['tic'].isin(selected_tickers)].copy()
offset = 15*100*10 +2
subset_df = subset_df.sort_values(["date", "tic"], ignore_index = True)
subset_df = subset_df.iloc[offset:]
print(len(subset_df))
print(subset_df.head())
print(subset_df.tail())
# Sort and reset
df = subset_df.sort_values(['tic', 'date']).reset_index(drop=True)
df

1365
            date    open     high     low   close    volume    tic
15002 2017-09-08  160.86  161.150  158.53  158.63  28611535   AAPL
15003 2017-09-08  979.10  979.880  963.47  965.90   2605322   AMZN
15004 2017-09-08  173.90  175.815  173.57  175.50   3736383  BRK.B
15005 2017-09-08  949.70  950.700  940.01  941.41    999916  GOOGL
15006 2017-09-08  159.09  160.690  157.52  159.66  10088630     HD
            date    open    high     low   close    volume   tic
16362 2018-02-07  229.58  234.97  226.70  228.80  20073316  NVDA
16363 2018-02-07   82.30   83.39   81.82   81.84  11851841    PG
16364 2018-02-07  222.33  229.29  221.51  225.82   5589154   UNH
16365 2018-02-07  120.09  122.46  119.21  119.65   9293996     V
16366 2018-02-07   78.44   79.41   76.92   76.94  21994450   XOM


,date,open,high,low,close,volume,tic
0,2017-09-08,160.86,161.15,158.5300,158.63,28611535,AAPL
1,2017-09-11,160.50,162.05,159.8900,161.50,31580798,AAPL
2,2017-09-12,162.61,163.96,158.7700,160.86,71714046,AAPL
3,2017-09-13,159.87,159.96,157.9100,159.65,44907361,AAPL
4,2017-09-14,158.99,159.40,158.0900,158.28,23760749,AAPL
...,...,...,...,...,...,...,...
1360,2018-02-01,87.50,89.25,87.3500,89.07,17971012,XOM
1361,2018-02-02,85.13,86.01,82.9978,84.53,29822144,XOM
1362,2018-02-05,83.28,83.99,78.1300,79.72,30452693,XOM
1363,2018-02-06,78.51,80.35,76.9000,78.35,36262761,XOM


## **2. Set Split Train & Test Date Data**

In [ ]:
from FinRl_Mods.feature_engineering import process_features

# --- Step 2: Feature Engineering ---
processed = process_features(df)

# --- Step 3: Environment Init ---
split_ratio = 0.8  #@param {type:"number"}
split_idx = int(len(df) * split_ratio)

TRAIN_START = df['date'].iloc[0]
TRAIN_END = df['date'].iloc[split_idx - 1]
TRADE_START = df['date'].iloc[split_idx]
TRADE_END = df['date'].iloc[-1]


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Successfully added technical indicators
✅ Processed features ready.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## **3. Train**

In [ ]:
from FinRl_Mods.env_initializer import init_stock_environment
from FinRl_Mods.agent_initializer import train_agent
# NB: 2-3 min
e_train_gym, e_trade_gym = init_stock_environment(processed, TRAIN_START, TRAIN_END, TRADE_START, TRADE_END)

# --- Step 4: Train Agent ---
trained_model = train_agent(e_train_gym, model_type="ppo", total_timesteps=50_000)



✅ Gym environments initialized.
{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Logging to results/ppo


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/finrl/meta/env_stock_trading/env_stocktrading.py:189: RuntimeWarning: divide by zero encountered in scalar floor_divide
  available_amount = self.state[0] // (
/usr/local/lib/python3.12/dist-packages/finrl/meta/env_stock_trading/env_stocktrading.py:189: RuntimeWarning: divide by zero encountered in scalar floor_divide
  available_amount = self.state[0] // (
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ret

day: 58, episode: 10
begin_total_asset: 1000000.00
end_total_asset: 756576.90
total_reward: -243423.10
total_cost: 7769.29
total_trades: 690
Sharpe: 2.407


...

day: 58, episode: 860
begin_total_asset: 1000000.00
end_total_asset: 107.02
total_reward: -999892.98
total_cost: 6908.24
total_trades: 654
Sharpe: 2.086


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/finrl/meta/env_stock_trading/env_stocktrading.py:189: RuntimeWarning: divide by zero encountered in scalar floor_divide
  available_amount = self.state[0] // (
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/finrl/meta/env_stock_trading/env_stocktrading.py:189: RuntimeWarning: divide by zero encountered in scalar floor_divid

-----------------------------------------
| time/                   |             |
|    fps                  | 379         |
|    iterations           | 25          |
|    time_elapsed         | 134         |
|    total_timesteps      | 51200       |
| train/                  |             |
|    approx_kl            | 0.009381793 |
|    clip_fraction        | 0.114       |
|    clip_range           | 0.2         |
|    entropy_loss         | -19         |
|    explained_variance   | 0.616       |
|    learning_rate        | 0.00025     |
|    loss                 | 430         |
|    n_updates            | 240         |
|    policy_gradient_loss | -0.00668    |
|    reward               | -0.26757708 |
|    reward_max           | 183.14319   |
|    reward_mean          | -1.0458612  |
|    reward_min           | -195.60167  |
|    std                  | 1.04        |
|    value_loss           | 639         |
-----------------------------------------


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


✅ PPO model trained successfully.


## **4. Backtest**

In [ ]:
from finrl.plot import backtest_stats, get_baseline
from finrl.agents.stablebaselines3.models import DRLAgent
import matplotlib.pyplot as plt

# --- Step 5: Backtest ---
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(model=trained_model, environment=e_trade_gym)

perf_stats_all_ppo = backtest_stats(account_value=df_account_value_ppo)
perf_stats_all_ppo = pd.DataFrame(perf_stats_all_ppo)
# perf_stats_all_ppo.to_csv(
#     "./" + config.RESULTS_DIR + "/perf_stats_all_ppo_" + now + ".csv"
# )

def calc_cum_return(df):
    grouped = df.groupby('tic')
    selected_cum_ret = grouped.apply(lambda x: x['open']/x['open'].iloc[0])
    selected_cum_ret = selected_cum_ret.reset_index()
    selected_cum_ret = selected_cum_ret.set_index('level_1')
    selected_cum_ret = selected_cum_ret.join(df[['date']])

    return selected_cum_ret

import matplotlib.pyplot as plt

def plot_cum_return(selected_cum_ret):
    fig, ax = plt.subplots()

    # 个股
    for key, grp in selected_cum_ret.groupby(['tic']):
        ax = grp.plot(ax=ax, kind='line', x='date', y='open', label=key)

    plt.show()
# Plot state data Cummulative Returns
cum_ret = calc_cum_return(processed)
plot_cum_return(cum_ret)

# TODO: Get Data localy
# print("==============Get Baseline Stats===========")
# baseline_df = get_baseline(ticker="^DJI", start=TEST_START_DATE, end=TEST_END_DATE)
# stats = backtest_stats(baseline_df, value_col_name="close")
# print(f"✅ Backtest & baseline stats complete")

df_result_ppo = df_account_value_ppo.set_index(df_account_value_ppo.columns[0])
result = df_result_ppo
result.columns = ["ppo"]

print("result: ", result)
result.to_csv("result.csv")


/usr/local/lib/python3.12/dist-packages/pandas_datareader/compat/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  PANDAS_VERSION = LooseVersion(pd.__version__)
/usr/local/lib/python3.12/dist-packages/pyfolio/pos.py:25: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/finrl/meta/env_stock_trading/env_stocktrading.py:189: RuntimeWarning: divide by zero encountered in scalar floor_divide
  available_amount = self.state[0] // (
/tmp/ipython-input-2726917775.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  selected_cum_ret = gr

hit end!
Annual return          0.514626
Cumulative returns     0.163659
Annual volatility      3.030857
Sharpe ratio           1.496311
Calmar ratio           1.161157
Stability              0.000055
Max drawdown          -0.443201
Omega ratio            1.518869
Sortino ratio          2.812098
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.463472
Daily value at risk   -0.363856
dtype: float64


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


result:                       ppo
date                    
2017-11-07  1.000000e+06
2017-11-08  9.999548e+05
2017-11-09  9.999875e+05
2017-11-10  9.999086e+05
2017-11-11  9.788748e+05
...                  ...
2018-02-02  1.171398e+06
2018-02-03  6.661313e+05
2018-02-04  6.661313e+05
2018-02-05  1.156073e+06
2018-02-06  1.163659e+06

[92 rows x 1 columns]


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
